In [78]:
import torch
import pandas as pd
from torch import nn

# Test trained neural network
use case ieee39, under the base of devise ($S_{base} = 1000 MVA$)

## Load network and norm data

In [79]:
# Load frequency model
fnn = torch.jit.load('net_fnadir.pt')
# Load vsg power model
pnn = torch.jit.load('net_Ppeak.pt')

In [80]:
# load norm dadta
fnorm = pd.read_csv('fnorm.csv')
pnorm = pd.read_csv('pnorm.csv')
norm = {'fnorm': fnorm, 'pnorm': pnorm }

In [81]:
fnorm

,M,D,Fg,Rg,fnadir
0,5.546693,1.18807,10.502496,19.996914,-0.079710
1,0.665088,0.39706,3.747091,2.903126,0.020099


In [82]:
pnorm

,M,D,Fg,Rg,Mvsg,Dvsg,Ppeak
0,5.546693,1.18807,10.502496,19.996914,2.511393,1.503477,0.344717
1,0.665088,0.39706,3.747091,2.903126,1.448681,0.863440,0.171101


## Test prediction results

### frequency prediction

In [83]:
fmean = fnorm.iloc[0].values
fstd = fnorm.iloc[1].values

In [84]:
xf = [5, 1, 10, 20]   
xf = (xf - fmean[0:4]) / fstd[0:4]

xf = torch.tensor(xf, dtype=torch.float32)
xf

tensor([-0.8220, -0.4737, -0.1341,  0.0011])

In [85]:
delta_Pe = -0.1

delta_f = fnn(xf).detach().numpy() * fstd[4] + fmean[4]

delta_f *= delta_Pe

delta_f

array([0.00833493], dtype=float32)

### vsg power prediction

In [86]:
pmean = pnorm.iloc[0].values
pstd = pnorm.iloc[1].values

In [87]:
xp = [5, 1, 10, 20, 4, 2]

xp = (xp - pmean[0:6]) / pstd[0:6]

xp = torch.tensor(xp, dtype=torch.float32)

xp


tensor([-0.8220, -0.4737, -0.1341,  0.0011,  1.0276,  0.5751])

In [88]:
delta_Pvsg = pnn(xp).detach().numpy() * pstd[6] + pmean[6]

delta_Pvsg *= delta_Pe

delta_Pvsg

array([-0.05625215], dtype=float32)